In [8]:

from radiant_mlhub import Dataset

bbox = [28.529402, -1.753992, 30.328979, 1.359627]

dataset = Dataset.fetch('ref_landcovernet_af_v1')
print(dataset.collections)
print(dataset.collections.source_imagery)
print(dataset.collections.labels)

collection_filter = dict(
    ref_landcovernet_af_v1_source_sentinel_2=['B02', 'B03', 'B04', 'B08', 'CLD'],
    ref_landcovernet_af_v1_labels=[]
)




[<Collection id=ref_landcovernet_af_v1_source_sentinel_2>, <Collection id=ref_landcovernet_af_v1_source_sentinel_1>, <Collection id=ref_landcovernet_af_v1_source_landsat_8>, <Collection id=ref_landcovernet_af_v1_labels>]
[<Collection id=ref_landcovernet_af_v1_source_sentinel_2>, <Collection id=ref_landcovernet_af_v1_source_sentinel_1>, <Collection id=ref_landcovernet_af_v1_source_landsat_8>]
[<Collection id=ref_landcovernet_af_v1_labels>]


In [9]:
catalog_size = dataset.stac_catalog_size
estimated_dataset_size = dataset.estimated_dataset_size
print("Catalog Size: ", catalog_size) 
print("Estimated Dataset Size: ", estimated_dataset_size) 

Catalog Size:  104896359
Estimated Dataset Size:  678690171091


In [ ]:
from datetime import datetime as dt

filters = dict(
    sen12floods_s2_source=['B02', 'B03', 'B04', 'B08'],   # Red, Green, Blue, NIR
    sen12floods_s2_labels=['labels', 'documentation'],
)

dataset.download(bbox=bbox, output_dir="./data/RadiantEarth/")
